In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [2]:
class recomm:
    def __init__(self): #输入基本参数（决定模型尺寸）
        self.variables=["product number","periods","display numbers","customer numbers"]
        print("please input {} :".format(self.variables[0])) #产品数量
        self.n=int(input())
        print("please input {} :".format(self.variables[1])) #一天来店人数
        self.T=int(input())
        print("please input {} :".format(self.variables[2])) #展示的产品数量
        self.K=int(input())
        print("please input {} :".format(self.variables[3])) #顾客种类数
        self.m=int(input())
        self.cvr=np.zeros((self.m,self.n))
        self.ctr=np.zeros((self.m,self.n))
        self.price=np.zeros(self.n)
        self.invent=np.zeros(self.n)
        self.cusprob=np.zeros(self.m)
        self.t=0
        self.Type=0
        self.c=0
        self.A=0
        self.b=0
        self.purchase=0
        self.tag=0
        self.rest=0
        self.duAl=0
        self.deCision=[]
        self.sorted_deCision=[]
        self.reCommend=[]
    ##输入高级参数
    def cvr_input(self): #cvr
            for i in range(self.m):
                for j in range(self.n):
                    print("please input cvr for  product {} of customer type {} :".format(j+1,i+1))
                    self.cvr[i,j]=float(input())
                    print(self.cvr)
    def ctr_input(self): #ctr
            for i in range(self.m):
                for j in range(self.n):
                    print("please input ctr for  product {} of customer type {} :".format(j+1,i+1))
                    self.ctr[i,j]=float(input())
                    print(self.ctr)   
    def price_input(self): #商品价格
            for i in range(self.n):
                print("please input price of  product {} :".format(i+1))
                self.price[i]=float(input())
                print(self.price)
    def inventory_input(self): #预库存
            for i in range(self.n):
                print("please input inventory of  product {} :".format(i+1))
                self.invent[i]=float(input())
                print(self.invent)
    def cusprob_input(self): #顾客来访的概率
            for i in range(self.m):
                print("please input probability of customer type {} :".format(i+1))
                self.cusprob[i]=float(input())
                print(self.cusprob)
    def purchaseprob_generation(self): #根据ctr和cvr生成顾客的商品购买概率矩阵
            self.purchase=np.zeros((self.m,self.n))
            for i in range(self.m):
                for j in range(self.n):
                    self.purchase[i,j]=self.ctr[i,j]*self.cvr[i,j]
            print("the purchase probability matrix is: :")
            print(self.purchase)
    def initialize(self): #初始化模型的参数设置
        self.cvr_input()
        self.ctr_input()
        self.price_input()
        #self.inventory_input()
        self.cusprob_input()
        self.purchaseprob_generation()
    def generate_c(self): #生成目标函数的系数
        self.c=np.zeros((1+self.m)*self.n)
        order=0
        for i in range(1+self.m):
            for j in range(self.n):
                if i==0:
                    self.c[order]=self.purchase[self.Type-1,j]*self.price[j]
                    order+=1
                if i >0:
                    self.c[order]=self.purchase[i-1,j]*self.price[j]*self.rest*self.cusprob[i-1]
                    order+=1
    def generate_A(self): #生成约束的系数
        self.A=np.zeros((self.m+1+self.n,(1+self.m)*self.n))
        for i in range(self.m+1):
            for j in range(self.n):
                self.A[i,self.n*i+j]=1
        for i in range(self.n):
            for j in range(self.m+1):
                if j==0:
                    self.A[self.m+1+i,j*self.n+i]=self.purchase[self.Type-1,i]
                if j>0:
                    self.A[self.m+1+i,j*self.n+i]=self.purchase[j-1,i]*self.rest*self.cusprob[j-1]
    def generate_b(self): #生成约束的值
        self.b=np.zeros(self.m+1+self.n)
        for i in range(self.m+1):
            self.b[i]=self.K
        for i in range(self.n):
            self.b[i+self.m+1]=self.invent[i]
    def sOrt(self): # 根据对偶的更新后的真实价格排序并推荐
        for i in range(self.n):
            self.deCision.append((self.price[i]-self.duAl[i])*self.purchase[self.Type-1,i])
        for j in range(self.K):
            for i in range(self.n):
                if self.deCision[i]==max(self.deCision):
                    if self.invent[i] != 0:
                        print("推荐第{}种商品".format(i+1))
                        self.reCommend.append(i+1)
                        self.deCision[i]=0
                        break
        self.deCision=[]
    def LP(self,t,Type,invent):  #进行规划求解，输入的有：1.这是今天第几个客人，2.来店的客人种类，3.[商品1的库存，商品2的库存]
        self.t=t
        self.Type=int(Type)
        self.invent=invent
        self.rest=int(self.T-self.t)
        #print(self.tag)
        self.generate_c()
        self.generate_A()
        self.generate_b()
        print(self.c)
        print(self.A)
        print(self.b)
        a=gp.Model("basic")
        x=a.addMVar((1+self.m)*self.n,name='x')
        a.setObjective(self.c@x, GRB.MAXIMIZE)
        a.addConstr(self.A@x <= self.b, name='cons')
        a.optimize()
        self.duAl=a.pi[-self.n:]   # 储存库存约束的对偶即库存的影子价格
        self.sOrt()

In [3]:
a=recomm()  # 创建类
a.initialize()   #输入模型的参数设置

please input product number :


 2


please input periods :


 100


please input display numbers :


...

[8. 5.]
please input probability of customer type 1 :


 0.6


[0.6 0. ]
please input probability of customer type 2 :


 0.4


[0.6 0.4]
the purchase probability matrix is: :
[[0.8 0.1]
 [0.4 0.5]]


In [9]:
t=1             # 本次交易为第t次
customer=1      # 顾客为第1种
invent=[40,40]  # 实时库存水平
a.LP(t,customer,invent)   # 进行线性规划并推荐


[  3.2    2.5  318.72  24.9  106.24  83.  ]
[[ 1.    1.    0.    0.    0.    0.  ]
 [ 0.    0.    1.    1.    0.    0.  ]
 [ 0.    0.    0.    0.    1.    1.  ]
 [ 0.4   0.   39.84  0.   13.28  0.  ]
 [ 0.    0.5   0.    4.98  0.   16.6 ]]
[ 1.  1.  1. 40. 40.]
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x609c5b9d
Coefficient statistics:
  Matrix range     [4e-01, 4e+01]
  Objective range  [3e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 4 rows and 4 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2816000e+02   1.690000e+00   0.000000e+00      0s
       1    4.0450000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal o

In [ ]:
##使用说明：先使用a=recomm()  # 创建类
#再使用a.initialize()   #输入模型的参数设置
#每次需要更新t=1             # 本次交易为第t次
#在这里输入顾客类型customer=1      # 顾客为第1种
#输入两种库存invent=[40,40]  # 实时库存水平
#执行线性规划得到推荐。推荐会打印出来a.LP(t,customer,invent)   # 进行线性规划并推荐
#并且每次的推荐还储存在a.reCommend列表